In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle
from numpy.random import RandomState
from sklearn import metrics
from sklearn.utils import shuffle
import os

In [10]:
# Functions for importing & cleaning labeled items
def lower(s):
    return s.lower()

def imports(filename):
    imp = pd.read_pickle(filename)
    imp = imp.drop_duplicates()
    imp['sent_clean'] = imp['sentence'].str.replace('http\S+|www.\S+|pic.twitter.com\S+', '', case=False)
    #imp['sent_clean'] = imp['sentence'].replace('[^A-Za-z0-9 ]+','',regex=True)
    imp['sent_clean'] = imp['sent_clean'].apply(lower)
    return imp

In [4]:
dallas_sentence_scores = pd.read_csv('sent_scores_df.tsv',delimiter='\t')
dallas_sentence_scores['max_prob_label'] = dallas_sentence_scores[['disagree',
                                                                   'neutral',
                                                                   'agree']].idxmax(axis=1)

In [5]:
dallas_sentence_scores.head()

,Unnamed: 0,round,batch,sent_id,disagree,neutral,agree,sentence,max_prob_label
0,0,1,0,t0,0.004241,0.260963,0.734797,Warmer-than-normal sea surface temperatures ar...,agree
1,1,1,0,t1,0.001548,0.996214,0.002239,We will continue to rely in part on fossil fue...,neutral
2,2,1,0,t10,0.001440,0.996503,0.002057,The actual rise in sea levels measured only 1....,neutral
3,3,1,0,t11,0.996815,0.001588,0.001596,Claims of global warming have been greatly exa...,disagree
4,4,1,0,t12,0.035201,0.959757,0.005042,The Intergovernmental Panel on Climate Change ...,neutral


In [7]:
dallas_sentence_scores.to_pickle('dallas_sent_scores.pkl')

In [11]:
sentence_df = imports('dallas_sent_scores.pkl')

In [12]:
sentence_df.head(10)

,Unnamed: 0,round,batch,sent_id,disagree,neutral,agree,sentence,max_prob_label,sent_clean
0,0,1,0,t0,0.004241,0.260963,0.734797,Warmer-than-normal sea surface temperatures ar...,agree,warmer-than-normal sea surface temperatures ar...
1,1,1,0,t1,0.001548,0.996214,0.002239,We will continue to rely in part on fossil fue...,neutral,we will continue to rely in part on fossil fue...
2,2,1,0,t10,0.001440,0.996503,0.002057,The actual rise in sea levels measured only 1....,neutral,the actual rise in sea levels measured only 1....
3,3,1,0,t11,0.996815,0.001588,0.001596,Claims of global warming have been greatly exa...,disagree,claims of global warming have been greatly exa...
4,4,1,0,t12,0.035201,0.959757,0.005042,The Intergovernmental Panel on Climate Change ...,neutral,the intergovernmental panel on climate change ...
5,5,1,0,t13,0.590718,0.335917,0.073365,Evidence now leans against global warming resu...,disagree,evidence now leans against global warming resu...
6,6,1,0,t14,0.784720,0.209294,0.005986,Flooding has been going on forever.,disagree,flooding has been going on forever.
7,7,1,0,t15,0.002637,0.027762,0.969601,Simply reducing emissions will not sufficientl...,agree,simply reducing emissions will not sufficientl...
8,8,1,0,t16,0.072546,0.501655,0.425799,Climate deniers blame global warming on aliens...,neutral,climate deniers blame global warming on aliens...
9,9,1,0,t17,0.245913,0.278034,0.476053,"Global warming is inevitably going to be, at b...",agree,"global warming is inevitably going to be, at b..."


# Create train / val / test split

In [13]:
all_train = sentence_df[['round','batch','sent_id','disagree','neutral','agree',
                        'max_prob_label','sent_clean']]

print(len(all_train), len(all_train[all_train['max_prob_label']=='agree']), 
      len(all_train[all_train['max_prob_label']=='neutral']),
     len(all_train[all_train['max_prob_label']=='disagree']))

2050 783 863 404


In [15]:
sub_train = all_train[all_train['sent_clean'].str.\
                      contains("climate|change|global|warming")==True]
print(len(sub_train))

1381


In [19]:
#randomly select tweets that go to validation set or training set
all_train = shuffle(all_train,random_state=123)
all_train = all_train.drop_duplicates('sent_clean')
num_sents = len(all_train)

print(num_sents)
infl_val_agr = all_train[all_train['max_prob_label']=='agree']
infl_val_neut = all_train[all_train['max_prob_label']=='neutral']
infl_val_dis = all_train[all_train['max_prob_label']=='disagree']
print(len(infl_val_agr), len(infl_val_neut), len(infl_val_dis))

train_pct = .9

2042
779 862 401


In [ ]:
training_data = all_train[:int(num_sents*train_pct)]

val_data = all_train[int(num_sents*train_pct):]

print("Num total sentences", num_ses,
      "\n Num train tweets", len(training_data), 
      "\n Num validation tweets", len(val_tweets), 
      "\n Num test tweets", len(test_tweets))